<a href="https://colab.research.google.com/github/mu06905/GPU-Accelerated-Programming-in-Cuda-2023/blob/main/Week3/Assignment1Q2_Sum_of_Array.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-x_1xdw2o
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-x_1xdw2o
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=7d4c5816d2dfedcfc3b119bac26744cb375d3c61d955414b83cfa9f2658cb2d0
  Stored in directory: /tmp/pip-ephem-wheel-cache-n7im3tp7/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [3]:
%%cu
#include <stdio.h>
#include <time.h>

#define CUDA_CALL(x) do { if((x)!=cudaSuccess) { \
    printf("Error at %s:%d\n",__FILE__,__LINE__);\
    return EXIT_FAILURE;}} while(0)
#define CURAND_CALL(x) do { if((x)!=CURAND_STATUS_SUCCESS) { \
    printf("Error at %s:%d\n",__FILE__,__LINE__);\
    return EXIT_FAILURE;}} while(0)

inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
  if (err != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
  }
  return err;
}
__global__ void sum_device(int* a, int* sum, const int N) {
	int i = threadIdx.x + blockDim.x * blockIdx.x;
	if (i<N)
		(*sum) += a[i];	
	//else 
	//	printf("i > N -> %3d in block: %d\n", i, blockIdx.x);
}
//this is an incorrect implementation of the task as each thread accesses the shared variable *sum at the same
//time resulting in race condition that is why the result on the gpu is not correct
//todo: figure out why differnt execution configurations give different results
void sum_host(int* a, int* sum, const int N) {
  clock_t clk;

	clk = clock();
	for(int i=0; i<N; ++i)
	  *sum += a[i];

	clk = clock() - clk;
	double dt = (((double)clk) / CLOCKS_PER_SEC) * 1000;
	printf("Total time on CPU: %f msecs\n", dt);

}


int main() { 
	int* h_a = 0;
	int sum = 0;
  int* sum_d = 0;
	
	int* d_a = 0;
	int* d_sum;

	const int N = 1000; 
	int numThreadsPerBlock =  128;
	int numBlocksPerGrid = ceilf(N/numThreadsPerBlock + 1);
	printf("Num threads per block: %3d\n", numThreadsPerBlock);
	printf("Num blocks per grid: %3d\n", numBlocksPerGrid);

	size_t size = N * sizeof(int);

	//allocate host memory
	h_a = (int*)malloc(size);	 

	//initialize a
	for(int i=0;i<N;++i) {
		h_a[i] = i+1;
	}
	//calculate on host 
	sum_host(h_a, &sum, N); 
	printf("Sum host: %d\n", sum); 

  cudaEvent_t start, stop;
	cudaEventCreate(&start);
	cudaEventCreate(&stop);

	//allocate device memory 
	checkCudaErr(cudaMalloc((void**)&d_a, size),"cudamalloc");
  checkCudaErr(cudaMalloc(&d_sum, sizeof(int)),"cudamalloc");
	
	//copy host data to device memory
	checkCudaErr(cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice),"cudamemcopy"); 


	checkCudaErr(cudaEventRecord(start),"event start");
	sum_device<<<numBlocksPerGrid, numThreadsPerBlock>>>(d_a, d_sum, N);
	CUDA_CALL(cudaDeviceSynchronize());
  checkCudaErr(cudaEventSynchronize(stop),"event synchronize");

  

	//copy result from device to host
	checkCudaErr(cudaMemcpy(&sum_d, d_sum, sizeof(int), cudaMemcpyDeviceToHost),"cuda memcopy");
	checkCudaErr(cudaEventRecord(stop),"event stop");


  printf("Sum (device): %d\n",sum_d);

 
 	float dt = 0;
	checkCudaErr(cudaEventElapsedTime(&dt, start, stop),"cuda elapsed time");
	printf("Total time on GPU: %f msecs\n", dt);

	//delete data allocated on device
	cudaFree(d_a);

	//delete host memory
	free(h_a);

	cudaDeviceReset();
	return 0;
}

CUDA Runtime error at cuda elapsed time: device not ready
Num threads per block: 128
Num blocks per grid:   8
Total time on CPU: 0.004000 msecs
Sum host: 500500
Sum (device): 609
Total time on GPU: 0.000000 msecs

